In [31]:
from sqlalchemy import create_engine
import pandas as pd
import sqlalchemy
import dtale

%config IPCompleter.greedy=True

core_address = 'analysts-dwh.cabify-data.com'
core_port = '5439'

f = open("/Users/santiagocanas/Downloads/redshift.txt", mode = "r")
file = f.readlines()

core_username = str(file[0].strip())
core_password = str(file[1].strip())
core_dbname = 'dwh'

dwh_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(
                        username = core_username,
                        password = core_password,
                        ipaddress = core_address,
                        port = core_port,
                        dbname = core_dbname))

dwh_cnx = create_engine(dwh_str)

In [32]:
sql = '''
select
    pid,
    user_name,
    DATEDIFF('minute',starttime, current_timestamp::timestamp(0)) as running_time,
    cast(starttime as timestamp(0)) as starttime,
    status,
    query
from
    stv_recents
where
    status = 'Running';'''
status = pd.read_sql_query(sql, dwh_cnx)
status

,pid,user_name,running_time,starttime,status,query


In [33]:
sql = '''
select pg_terminate_backend(10058)'''
kill = pd.read_sql_query(sql, dwh_cnx)
kill

,pg_terminate_backend
0,0


In [37]:
sql = '''
select 
    distinct
        table_name,
        column_name, 
        table_catalog, 
        table_schema
from 
	information_schema.columns
where 
    table_name = 'fin_fac_paypalwinloss'
'''

search = pd.read_sql_query(sql, dwh_cnx)
search

,table_name,column_name,table_catalog,table_schema
0,fin_fac_paypalwinloss,ad_inserted_at,dwh,datawarehouse
1,fin_fac_paypalwinloss,cd_chargeback_reason_code,dwh,datawarehouse
2,fin_fac_paypalwinloss,cd_disputed_currency,dwh,datawarehouse
3,fin_fac_paypalwinloss,cd_fee_currency,dwh,datawarehouse
4,fin_fac_paypalwinloss,cd_gross_transaction_currency,dwh,datawarehouse
5,fin_fac_paypalwinloss,cd_seller_protection_currency,dwh,datawarehouse
6,fin_fac_paypalwinloss,ds_buyer_comments,dwh,datawarehouse
7,fin_fac_paypalwinloss,ds_buyer_email_address,dwh,datawarehouse
8,fin_fac_paypalwinloss,ds_buyer_first_name,dwh,datawarehouse
9,fin_fac_paypalwinloss,ds_buyer_last_name,dwh,datawarehouse


In [35]:
sql = '''
select 
    cd_disputed_currency,
    ds_case_reason,
    ds_case_status,
    ds_case_type,
    ds_settlement_type,
    qt_disputed_amount_local,
    qt_disputed_amount_usd,
    qt_fee_amount_local,
    qt_fee_amount_usd
from 
    fin_fac_paypalwinloss_all_states pp
where
    pp.id_case_id = 'PP-D-96354211' 
'''
data = pd.read_sql_query(sql, dwh_cnx)
data

,cd_disputed_currency,ds_case_reason,ds_case_status,ds_case_type,ds_settlement_type,qt_disputed_amount_local,qt_disputed_amount_usd,qt_fee_amount_local,qt_fee_amount_usd
0,EUR,Unauthorized,Case Closed,Bank Return,Reversal,93.68,114.15,1.84,2.24
1,EUR,Unauthorized,Being Reviewed By PayPal,Bank Return,Reversal,93.68,114.15,1.84,2.24
2,EUR,Unauthorized,Waiting For Seller's Response,Bank Return,None,93.68,114.15,1.84,2.24
3,EUR,Unauthorized,Waiting For Seller's Response,Unauthorized Claim,None,93.68,114.15,1.84,2.24
4,EUR,Unauthorized,Waiting For Seller's Response,Unauthorized Claim,None,93.68,114.15,1.84,2.24
5,EUR,Unauthorized,Being Reviewed By PayPal,Bank Return,Reversal,93.68,114.15,1.84,2.24
6,EUR,Unauthorized,Being Reviewed By PayPal,Bank Return,Reversal,93.68,114.15,1.84,2.24
7,EUR,Unauthorized,Waiting For Seller's Response,Bank Return,None,93.68,114.15,1.84,2.24
8,EUR,Unauthorized,Being Reviewed By PayPal,Bank Return,Reversal,93.68,114.15,1.84,2.24
9,EUR,Unauthorized,Waiting For Seller's Response,Bank Return,None,93.68,114.15,1.84,2.24


In [36]:
sql = '''
with
cases_with_two_states AS (
select 
    id_case_id,
    COUNT(DISTINCT trim(lower(ds_final_case_outcome)))
from 
    datawarehouse.fin_fac_paypalwinloss_all_states
where
    ds_final_case_outcome is not null
group by
    1
having
    COUNT(DISTINCT trim(lower(ds_final_case_outcome))) > 1
)

select 
        pp.id_case_id,
        pp.dt_file_date, 
        pp.tm_case_closure_date_utc, 
        pp.ds_final_case_outcome,
        pp.cd_disputed_currency,
        pp.ds_case_reason,
        pp.ds_case_status,
        pp.ds_case_type,
        pp.ds_settlement_type,
        pp.qt_disputed_amount_local,
        pp.qt_disputed_amount_usd,
        pp.qt_fee_amount_local,
        pp.qt_fee_amount_usd
from 
    datawarehouse.fin_fac_paypalwinloss_all_states pp
        inner join 
    cases_with_two_states ts on pp.id_case_id = ts.id_case_id
where
    pp.ds_final_case_outcome is not null
order by 
    1,2
'''
data = pd.read_sql_query(sql, dwh_cnx)
data

,id_case_id,dt_file_date,tm_case_closure_date_utc,ds_final_case_outcome,cd_disputed_currency,ds_case_reason,ds_case_status,ds_case_type,ds_settlement_type,qt_disputed_amount_local,qt_disputed_amount_usd,qt_fee_amount_local,qt_fee_amount_usd
0,PP-D-101488934,2021-02-14,NaT,Loss,EUR,Charge not recognized,Case Closed,Chargeback,Adjustment,11.49,13.53,0.34,0.41
1,PP-D-101488934,2021-04-30,2021-04-30 11:48:19,Win,EUR,Charge not recognized,Case Closed,Chargeback,Adjustment,11.49,13.53,0.34,0.41
2,PP-D-101488937,2021-02-14,NaT,Loss,EUR,Charge not recognized,Case Closed,Chargeback,Adjustment,16.12,17.64,0.41,0.45
3,PP-D-101488937,2021-04-30,2021-04-30 11:48:22,Win,EUR,Charge not recognized,Case Closed,Chargeback,Adjustment,16.12,17.64,0.41,0.45
4,PP-D-101488942,2021-02-14,NaT,Loss,EUR,Charge not recognized,Case Closed,Chargeback,Adjustment,10.00,10.86,0.32,0.34
5,PP-D-101488942,2021-04-30,2021-04-30 11:48:27,Win,EUR,Charge not recognized,Case Closed,Chargeback,Adjustment,10.00,10.86,0.32,0.34
6,PP-D-101488947,2021-02-14,NaT,Loss,EUR,Charge not recognized,Case Closed,Chargeback,Adjustment,11.91,13.00,0.34,0.38
7,PP-D-101488947,2021-04-30,2021-04-30 11:48:21,Win,EUR,Charge not recognized,Case Closed,Chargeback,Adjustment,11.91,13.00,0.34,0.38
8,PP-D-101491121,2021-02-14,NaT,Loss,EUR,Charge not recognized,Case Closed,Chargeback,Adjustment,0.28,0.34,0.42,0.50
9,PP-D-101491121,2021-04-30,2021-04-30 12:07:43,Win,EUR,Charge not recognized,Case Closed,Chargeback,Adjustment,0.28,0.34,0.42,0.50


In [8]:
data = pd.read_excel('/Users/santiagocanas/Downloads/fraq.xlsx')
data

,Date Selector,Aggregation Framework,Assigned_to_email,Date Selector,Solved_by_email,Source,Status,Subcategory,Type,item_age,category,Category,id,solved_at,source,status,subcategory,type,# Cases
0,2021-05-02,PENDING,NaN,5/6/2021 9:00:24 a.m.,NaN,SARUMAN,PENDING,UNASSIGNED,DRIVER,18,CASE_CATEGORY_UNASSIGNED,UNASSIGNED,12550,NaN,CASE_SOURCE_SARUMAN,CASE_REVIEW_STATUS_PENDING,CASE_SUBCATEGORY_UNASSIGNED,CASE_REVIEWABLE_TYPE_DRIVER,1
1,2021-05-02,PENDING,NaN,5/6/2021 9:00:24 a.m.,NaN,SARUMAN,PENDING,UNASSIGNED,DRIVER,18,CASE_CATEGORY_UNASSIGNED,UNASSIGNED,12549,NaN,CASE_SOURCE_SARUMAN,CASE_REVIEW_STATUS_PENDING,CASE_SUBCATEGORY_UNASSIGNED,CASE_REVIEWABLE_TYPE_DRIVER,1
2,2021-05-02,PENDING,NaN,5/6/2021 9:00:24 a.m.,NaN,SARUMAN,PENDING,UNASSIGNED,DRIVER,18,CASE_CATEGORY_UNASSIGNED,UNASSIGNED,12553,NaN,CASE_SOURCE_SARUMAN,CASE_REVIEW_STATUS_PENDING,CASE_SUBCATEGORY_UNASSIGNED,CASE_REVIEWABLE_TYPE_DRIVER,1
3,2021-05-02,PENDING,NaN,5/6/2021 9:00:24 a.m.,NaN,SARUMAN,PENDING,UNASSIGNED,DRIVER,18,CASE_CATEGORY_UNASSIGNED,UNASSIGNED,12544,NaN,CASE_SOURCE_SARUMAN,CASE_REVIEW_STATUS_PENDING,CASE_SUBCATEGORY_UNASSIGNED,CASE_REVIEWABLE_TYPE_DRIVER,1
4,2021-05-02,PENDING,NaN,5/6/2021 9:00:24 a.m.,NaN,SARUMAN,PENDING,UNASSIGNED,DRIVER,18,CASE_CATEGORY_UNASSIGNED,UNASSIGNED,12546,NaN,CASE_SOURCE_SARUMAN,CASE_REVIEW_STATUS_PENDING,CASE_SUBCATEGORY_UNASSIGNED,CASE_REVIEWABLE_TYPE_DRIVER,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1211,2021-05-02,PENDING,NaN,5/6/2021 9:00:55 a.m.,NaN,SARUMAN,PENDING,UNASSIGNED,DRIVER,18,CASE_CATEGORY_UNASSIGNED,UNASSIGNED,13818,NaN,CASE_SOURCE_SARUMAN,CASE_REVIEW_STATUS_PENDING,CASE_SUBCATEGORY_UNASSIGNED,CASE_REVIEWABLE_TYPE_DRIVER,1
1212,2021-05-02,PENDING,NaN,5/6/2021 9:00:55 a.m.,NaN,SARUMAN,PENDING,UNASSIGNED,DRIVER,18,CASE_CATEGORY_UNASSIGNED,UNASSIGNED,13811,NaN,CASE_SOURCE_SARUMAN,CASE_REVIEW_STATUS_PENDING,CASE_SUBCATEGORY_UNASSIGNED,CASE_REVIEWABLE_TYPE_DRIVER,1
1213,2021-05-02,PENDING,NaN,5/6/2021 9:00:55 a.m.,NaN,SARUMAN,PENDING,UNASSIGNED,DRIVER,18,CASE_CATEGORY_UNASSIGNED,UNASSIGNED,13820,NaN,CASE_SOURCE_SARUMAN,CASE_REVIEW_STATUS_PENDING,CASE_SUBCATEGORY_UNASSIGNED,CASE_REVIEWABLE_TYPE_DRIVER,1
1214,2021-05-02,PENDING,NaN,5/6/2021 9:00:55 a.m.,NaN,SARUMAN,PENDING,UNASSIGNED,DRIVER,18,CASE_CATEGORY_UNASSIGNED,UNASSIGNED,13813,NaN,CASE_SOURCE_SARUMAN,CASE_REVIEW_STATUS_PENDING,CASE_SUBCATEGORY_UNASSIGNED,CASE_REVIEWABLE_TYPE_DRIVER,1


In [9]:
data.columns

Index(['Date Selector', 'Aggregation Framework', 'Assigned_to_email',
       'Date Selector ', 'Solved_by_email', 'Source', 'Status', 'Subcategory',
       'Type', 'item_age', 'category', 'Category', 'id', 'solved_at', 'source',
       'status', 'subcategory', 'type', '# Cases'],
      dtype='object')

In [11]:
data[['Date Selector ','solved_at','item_age']].head(10)

,Date Selector,solved_at,item_age
0,5/6/2021 9:00:24 a.m.,NaN,18
1,5/6/2021 9:00:24 a.m.,NaN,18
2,5/6/2021 9:00:24 a.m.,NaN,18
3,5/6/2021 9:00:24 a.m.,NaN,18
4,5/6/2021 9:00:24 a.m.,NaN,18
5,5/6/2021 9:00:24 a.m.,NaN,18
6,5/6/2021 9:00:24 a.m.,NaN,18
7,5/6/2021 9:00:24 a.m.,NaN,18
8,5/6/2021 9:00:24 a.m.,NaN,18
9,5/6/2021 9:00:24 a.m.,NaN,18


In [21]:
data = pd.read_excel('/Users/santiagocanas/Downloads/unknown.xlsx')
data

,(F) Date Selection,Not Capture Aggregation Type,text field,Client type,Date Selection,Is Payment Capture?,P. transaction intent,Payment country,Payment Transaction Type,fk_paymenttransaction_type,...,Payment created at,Payment id,Payment state,sk_paymenttransaction,# Not Capture,Not Capture Local,Not Capture Selector,Not Capture USD,qt_amount_value_local,qt_amount_value_usd
0,2021-05-09,Unknown,$,private,2021-09-05,False,purchase,ES,Unknown,-1,...,2021-05-05,at6hmldcqesng994h0u4jjtok,declined,471306597,1,12.02,14.52,14.52,12.02,14.52
1,2021-05-09,Unknown,$,private,2021-09-05,False,purchase,ES,Unknown,-1,...,2021-03-05,bd1m57m8h7aalipiipi02jvb5,declined,471443676,1,11.58,13.99,13.99,11.58,13.99
2,2021-05-09,Unknown,$,private,2021-09-05,False,purchase,ES,Unknown,-1,...,2021-04-05,arksoaasey7d9v4mvkviu3zdu,declined,471229439,1,11.44,13.82,13.82,11.44,13.82
3,2021-05-09,Unknown,$,private,2021-09-05,False,purchase,ES,Unknown,-1,...,2021-04-05,acifu7bwwegn5guy78wctni7f,declined,471450947,1,13.97,16.88,16.88,13.97,16.88


In [22]:
data['Payment id'].head()

0    at6hmldcqesng994h0u4jjtok
1    bd1m57m8h7aalipiipi02jvb5
2    arksoaasey7d9v4mvkviu3zdu
3    acifu7bwwegn5guy78wctni7f
Name: Payment id, dtype: object